In [1]:
%load_ext autoreload
%load_ext line_profiler

In [2]:
%autoreload 1

In [3]:
import sys

# Allow importing game modules.
sys.path.append('../')

In [4]:
# Auto reload wordle libraries on edit -> save
%aimport wordle
%aimport solver

In [5]:
%aimport wordle_game.helper

In [6]:
import os
import cProfile, pstats
from multiprocessing import Pool
from functools import partial

import numpy as np
from tqdm import tqdm

In [7]:
wordlist = wordle.WordList()

In [8]:
wordlist.answers[0], wordlist.acceptable_guesses[0]

('aback', 'aahed')

In [9]:
wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

array([2, 1, 0, 0, 0])

In [10]:
wordle_game.helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

5

## Profiling

In [11]:
%timeit wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

5.27 µs ± 57.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit wordle_game.helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

753 ns ± 4.89 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%lprun -f wordle.generate_state wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

In [10]:
def profiler():
    for i in range(100000):
        wordle_game.helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

In [11]:
cProfile.runctx("profiler()", globals(), locals(), "Profile.prof")

In [12]:
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("cumtime").print_stats()

Wed Jan 26 21:44:29 2022    Profile.prof

         1497120 function calls in 0.496 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.496    0.496 {built-in method builtins.exec}
        1    0.000    0.000    0.496    0.496 <string>:1(<module>)
        1    0.040    0.040    0.496    0.496 2419803596.py:1(profiler)
   100000    0.027    0.000    0.456    0.000 {built-in method wordle_game.helper.generate_state}
   100000    0.019    0.000    0.429    0.000 helper.pyx:10(generate_state (wrapper))
   100000    0.327    0.000    0.410    0.000 helper.pyx:10(generate_state)
  1197116    0.083    0.000    0.083    0.000 helper.pyx:7(map_char)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# Generating all states

In [12]:
# state_space = np.empty((wordle.WORD_LEN, len(wordlist.answers)*len(wordlist.acceptable_guesses)), dtype=int)
state_space = np.zeros((len(wordlist.answers), len(wordlist.acceptable_guesses)), dtype=int)

for i, target in enumerate(tqdm(wordlist.answers)):
    for j, guess in enumerate(wordlist.acceptable_guesses):
        state_space[i, j] = wordle_game.helper.generate_state(target, guess)
        
# state_space = state_space.T
np.save('state_space.npy', state_space)

100%|█████████████████████████████████████████████████████████████████████████████| 2315/2315 [00:21<00:00, 106.69it/s]


In [13]:
i=0
j=0
wordlist.answers[i], wordlist.acceptable_guesses[j], wordle.generate_state(wordlist.answers[i], wordlist.acceptable_guesses[j])

('aback', 'aahed', array([2, 1, 0, 0, 0]))

In [14]:
state_space[i, j]

5

In [8]:
state_space = np.load('state_space.npy')

In [9]:
state_space.shape

(2315, 12972)

In [10]:
state_space.shape

(2315, 12972)

In [11]:
counts = wordle_game.helper.calculate_counts(state_space)

In [12]:
entropy = wordle_game.helper.calc_entropy(counts)

In [13]:
np.array(wordlist.acceptable_guesses)[np.argpartition(entropy, 3)[:3]]

array(['roate', 'raise', 'soare'], dtype='<U5')

In [20]:
wordlist.acceptable_guesses[np.argmax(entropy)]

'qajaq'

In [21]:
def convert_state(state):
    return state[0]+state[1]*3+state[2]*9+state[3]*27+state[4]*81

In [152]:
game = wordle.WordleGame(wordlist)
game.guess('soare')
guess_num = 8530

filter_mat = np.ones(state_space.shape[0], dtype=int)
while game.complete() == wordle.IN_PROGRESS:
    print(game)
    answers = solver.filter_answers(game)
    print("Answers", len(answers))
    if len(answers) == 1:
        print("Answer is", answers[0])
        game.guess(answers[0])
        print(game)
        break

    state = convert_state(game.state[game.round - 2])
    filter_mat[state_space[:, guess_num] != state] = 0
    
    #counts
    counts = calculate_counts(state_space, filter_mat)
    print("Count", counts.sum())
    # entropy
    entropy = calc_entropy(counts)
    
    print(np.array(wordlist.acceptable_guesses)[np.argpartition(entropy, 5)[:5]])
    # min entropy
    guess_num = np.argmin(entropy)
    print(entropy[guess_num])
    guess = wordlist.acceptable_guesses[guess_num]
    print(guess)
    game.guess(guess)


GYBBB
     
     
     
     
     

Answers 37


100%|██████████| 2315/2315 [00:00<00:00, 5411.87it/s] 


Count 479964
['punch' 'cloot' 'nicht' 'pinot' 'clipt']
-3.8740605612282875
cloot
GYBBB
BBGGB
     
     
     
     

Answers 7


100%|██████████| 2315/2315 [00:00<00:00, 26696.80it/s]


Count 90804
['spink' 'knosp' 'spunk' 'spank' 'upend']
-2.797096305432331
knosp
GYBBB
BBGGB
BGGYG
     
     
     

Answers 1
Answer is snoop
GYBBB
BBGGB
BGGYG
GGGGG
     
     



In [8]:
game = wordle.WordleGame(target='perky')

In [9]:
solve = solver.MaxEntropy()

In [10]:
game.add_state("soare","BBBYY")
print(game)
solve.guess(game)

BBBYY
     
     
     
     
     



'direr'

In [11]:
%lprun -f solve.guess solve.guess(game)

In [23]:
game.add_state("threw","BBGYB")
print(game)
solve.guess(game)

BBBYY
BBGYB
     
     
     
     



'dampy'

In [24]:
game.add_state("nerdy","BGGBG")
print(game)
solve.guess(game)

BBBYY
BBGYB
BGGBG
     
     
     



'abamp'

In [26]:
game.add_state("mercy","BGGBG")
print(game)
solve.guess(game)

BBBYY
BBGYB
BGGBG
BGGBG
     
     



'abamp'

In [27]:
solver.filter_answers(game)

['jerky', 'perky']

# Simulate

In [12]:
solutions = solve.simulate()
np.save('../data/maxentropy_solutions.npy', solutions)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2315/2315 [43:47<00:00,  1.14s/it]


In [ ]:
solutions = np.load('../data/maxentropy_solutions.npy')

In [15]:
unique, counts = np.unique(solutions, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  -1   24]
 [   2   22]
 [   3  883]
 [   4 1274]
 [   5  110]
 [   6    2]]


In [17]:
# Average number of guesses
np.sum(unique[1:]*counts[1:])/counts[1:].sum()

3.645133129637713

```python
cpu = os.cpu_count() - 1
cpu = cpu if cpu > 0 else cpu


results = []

for target in tqdm(wordlist.answers):
    with Pool(os.cpu_count() - 1) as pool:
        results.extend(pool.map(partial(wordle_game.helper.generate_state, target), wordlist.acceptable_guesses))
        pool.close()
        pool.join()
        
```